In [12]:
import os
from binance import Client
from datetime import timedelta,datetime
import pandas as pd
from tqdm import tqdm

In [24]:
API_KEY = os.environ.get('BINANCE_API_KEY')
SECRET_KEY = os.environ.get('BINANCE_SECRET_KEY')
BASE_DIR = "./../../../data/_raw"

In [14]:
client = Client(api_key=API_KEY,api_secret=SECRET_KEY)

In [15]:
def count_intervals(start_time, end_time, ms):
    interval_delta = timedelta(milliseconds=ms)
    current_time = start_time
    count = 0
    while current_time <= end_time:
        current_time += interval_delta
        count += 1
    return count


In [22]:
def create_csv_file(base_dir,symbol,interval,file_idx,data,override=False): 
    file_name = f"RAW_{symbol}_{interval}_{file_idx}.csv"
    file_path = os.path.join(base_dir,symbol,interval,file_name)
    if not os.path.exists(file_path) or (override is True and os.path.exists(file_path)):
        df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
        # df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.to_csv(file_path, index=False)

In [25]:
intervals = [
    ("1m",1000 * 60),    
    ("3m",1000 * 60 * 3),
    ("5m",1000 * 60 * 5),
    ("15m",1000 * 60 * 15),
    ("30m",1000 * 60 * 30),
    ("1h",1000 * 60 * 60 * 1),    
    ("2h",1000 * 60 * 60 * 2),
    ("4h",1000 * 60 * 60* 4),
    ("6h",1000 * 60 * 60 * 6),
    ("8h",1000 * 60 * 60 * 8),
    ("12h",1000 * 60 * 60 * 12),
    ("1d",1000 * 60 * 60 * 8),    
    ("3d",1000 * 60 * 60 * 24 * 3),
    ("1w",1000 * 60 * 60 * 24 * 3),
]
start_date = datetime(2017, 1, 1)
end_date = datetime(2023, 7, 21)
symbols = ["BTCUSDT"]
override = True
for symbol in symbols:
    for interval in intervals:
        _interval = interval[0]
        print("interval: ", _interval)
        approx = interval[1]
        approx_size = count_intervals(start_date,end_date,approx)
        print(approx_size)
        data = client.get_historical_klines_generator(
            symbol=symbol,
            interval=_interval,
            start_str="1 jan 2017",
            end_str="21 july 2023",
        )
        os.makedirs(os.path.join(BASE_DIR), exist_ok=True)
        os.makedirs(os.path.join(BASE_DIR,symbol), exist_ok=True)
        os.makedirs(os.path.join(BASE_DIR,symbol,_interval), exist_ok=True) 

        max_batch_size = 256
        num_rows_per_file = max_batch_size * 100
        file_rows = []
        file_num = 0
        for idx,item in tqdm(enumerate(data),total=approx_size):
            file_rows.append(item)
            if idx != 0 and idx % num_rows_per_file == 0:
                create_csv_file(base_dir=BASE_DIR,symbol=symbol,interval=_interval,file_idx=file_num,data=file_rows,override=override)
                file_rows = []
                file_num += 1

        if len(file_rows) > 0:
            create_csv_file(base_dir=BASE_DIR,symbol=symbol,interval=_interval,file_idx=file_num,data=file_rows,override=override)


interval:  1m
3444481


 90%|█████████ | 3107289/3444481 [40:53<04:26, 1266.36it/s]


interval:  3m
1148161


 58%|█████▊    | 665601/1148161 [08:49<04:31, 1776.89it/s]